In [200]:
import numpy as np
import random

In [201]:
alphabet = "abcdefghijklmnopqrstuvwxyz"

In [202]:
# if neg = False, vals will be from 0-1, else -1 - 1
def map_gen(neg=False, mul=1, dim=2, alphabet=alphabet):
    symbols = {}
    for char in alphabet:
        symbols[char] = np.random.rand(dim, dim)
    if neg:
        for char in alphabet:
            with np.nditer(symbols[char], op_flags=['readwrite']) as it:
                for x in it:
                    x[...] = 2 * x
                    if x > 1:
                        x[...] = 1 - x
    if mul != 1:
        for char in alphabet:
            with np.nditer(symbols[char], op_flags=['readwrite']) as it:
                for x in it:
                    x[...] = x * mul
    return symbols

In [203]:
map_gen(neg=-1, mul=10)

{'a': array([[ 1.64666355, -6.16818704],
        [ 2.18995239,  2.50262972]]),
 'b': array([[-2.39425695, -2.82366834],
        [ 8.67540942,  8.71672395]]),
 'c': array([[-3.07818776,  4.26864607],
        [ 3.59247934,  8.60389637]]),
 'd': array([[-6.20815005,  7.27249234],
        [ 2.75620541, -3.53457659]]),
 'e': array([[-2.03091975, -1.2632114 ],
        [ 8.72098406,  7.54836267]]),
 'f': array([[ 8.09822344, -8.21561394],
        [ 8.84918476,  7.59411315]]),
 'g': array([[-7.29078486, -8.99469213],
        [ 5.72248621, -8.39833075]]),
 'h': array([[ 3.51320437, -1.38377802],
        [-0.65705631, -2.40986458]]),
 'i': array([[-0.37089553, -9.75054716],
        [-1.94982566, -7.11877283]]),
 'j': array([[ 0.85355725, -9.51639547],
        [-5.3184371 , -7.67501286]]),
 'k': array([[ 1.2330365 , -4.59508526],
        [-9.87078287,  8.63059735]]),
 'l': array([[-8.26205082, -0.30234473],
        [ 1.72110991,  7.82483157]]),
 'm': array([[ 2.12243961,  4.84185266],
        [ 4

In [204]:
def wordMult(word):
  """Converts a word into its matrix form, by multiplying the symbol matrices."""
  if(len(word) == 1): return alphabet.get(word)
  mp = np.dot(alphabet.get(word[len(word)-2]), alphabet.get(word[len(word)-1]))
  if(len(word) == 2): 
    return mp
  for i in range(len(word)-2):
    mp = np.dot(alphabet.get(word[len(word)-i-3]), mp)
  return mp

def getTrace(arr):
  """Get the trace of a matrix (sum of the diagonal elements)."""
  return np.trace(arr)

def checkDensityValidity(Pl, Pr):
  """Check if the density matrices, Pl and Pr, are valid. The trace of their product must be 1."""
  mp = np.dot(Pl, Pr)
  return (getTrace(mp) == 1)

def phi(M, Pl, Pr):
  """Get the estimated frequency of a word. tr(Pl M Pr M*)"""
  M_cross = np.asmatrix(M).getH() # gets complex conjugate transpose
  mp = np.dot(Pr, M_cross)
  mp = np.dot(M, mp)
  mp = np.dot(Pl, M)
  # used to absolute value, but that was because our symbol matrices had negative values :D
  return getTrace(mp)
    
def grabValuesRandom(iterations, high, target):
  worstError = -1
  bestError = 1000
  bestPl = None
  bestPr = None
  bestEstimates = {}
  iter = 0
  for a in range(iterations):
    a1 = (random.random()*100) % high
    for b in range(iterations):
      b1 = (random.random()*100) % high
      for c in range(iterations):
        c1 = (random.random()*100) % high
        for d in range(iterations):
          d1 = (random.random()*100) % high
          for e in range(iterations):
            e1 = (random.random()*100) % high
            for f in range(iterations):
              f1 = (random.random()*100) % high
              for g in range(iterations):
                g1 = (random.random()*100) % high
                h=(1-(a1*e1+b1*g1+c1*f1))/d1
                m1 = np.array([[a1, b1],[c1, d1]])
                m2 = np.array([[e1, f1],[g1, h]])
                iter += 1
                if checkDensityValidity(m1, m2):
                  totalError = 0
                  estimates = {}
                  for z in target.keys():
                    totalError += (abs(phi(wordMult(z), m1, m2)-target[z])**2)
                    estimates[z] = phi(wordMult(z), m1, m2)
                  averageError = totalError / len(target.keys())  

                  if (averageError < bestError):
                    bestEstimates = estimates
                    bestError = averageError
                    bestPl = m1
                    bestPr = m2 
                  if (averageError > worstError):
                    worstError = averageError

  return bestError, bestEstimates, bestPl, bestPr, worstError
  
targetFrequencies = {"art": 0.15, "rat": 0.15, "at": 0.25, "a": 0.33, "tar": 0.05}
bestError = 1000
worstError = -1
bestMap = None
bestEstimates = {}
bestPl = None
bestPr = None
for i in range(100):
  print(f"Iteration: {i}.")
  alphabet = map_gen(alphabet="art")
  error, d, Pl, Pr, worst = grabValuesRandom(6, 1, targetFrequencies)
  if (error < bestError):
    bestEstimates = d
    bestError = error
    bestMap = alphabet
    bestPl = Pl
    bestPr = Pr
    print(f"Current best error average: {bestError}.")
  if (worst > worstError):
    worstError = worst

print("Best map:")
print(bestMap)
print("Best Pl:")
print(bestPl)
print("Best Pr:")
print(bestPr)
print("Best average error:")
print(bestError)
print("Worst average error:")
print(worstError)
for word in bestEstimates:
  print(f"Word: {word}.")
  print(f"Real freq: {targetFrequencies[word]}.")
  print(f"Calculated freq: {bestEstimates[word]}.")
  print(f"Deviance: {abs(targetFrequencies[word]-bestEstimates[word])}.\n")

Iteration: 0.
Current best error average: 0.02236845291281907.
Iteration: 1.
Current best error average: 0.020834412878091513.
Iteration: 2.
Iteration: 3.
Iteration: 4.
Current best error average: 0.009812695599076661.
Iteration: 5.
Iteration: 6.
Iteration: 7.
Iteration: 8.
Iteration: 9.
Iteration: 10.
Current best error average: 0.0006602128671610119.
Iteration: 11.
Iteration: 12.
Iteration: 13.
Iteration: 14.
Iteration: 15.
Iteration: 16.
Iteration: 17.
Iteration: 18.
Iteration: 19.
Iteration: 20.
Iteration: 21.
Iteration: 22.
Iteration: 23.
Iteration: 24.
Iteration: 25.
Iteration: 26.
Iteration: 27.
Iteration: 28.
Iteration: 29.
Iteration: 30.
Iteration: 31.
Iteration: 32.
Iteration: 33.
Iteration: 34.
Iteration: 35.
Iteration: 36.
Iteration: 37.
Iteration: 38.
Iteration: 39.
Iteration: 40.
Iteration: 41.
Iteration: 42.
Iteration: 43.
Iteration: 44.
Iteration: 45.
Iteration: 46.
Iteration: 47.
Iteration: 48.
Iteration: 49.
Iteration: 50.
Iteration: 51.
Iteration: 52.
Iteration: 53.
